## Environment setup

In [1]:
import glob, os, time
from datetime import datetime
from functools import partial
import importlib

import numpy as np

import tensorflow as tf

2023-04-06 22:00:07.427520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Networks definitions and adaptations

In [2]:
from architecture_definitions import *
 
dir_postfix = get_classic_nn_dir_postfix() 
    
print_Classic_NN()  

Classic NN definitions:
dense_layer1_size: 128
dense_layer2_size: 64
dense_layer3_size: 48
dense_layer4_size: 1
------------------------


### Training data set preparation

In [8]:
import io_functions as io
importlib.reload(io)

batchSize = 4096
nEpochs = 1

#trainDataDir = "/scratch_ssd/akalinow/ProgrammingProjects/MachineLearning/OMTF/data/18_12_2020/"   
trainDataDir = "/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/"
trainFileNames = glob.glob(trainDataDir+'OMTFHits_pats0x0003_oldSample_files_*_chunk_0.tfrecord.gzip')

trainDataDir = "/scratch_cmsse/alibordi/data/"
trainFileNames = glob.glob(trainDataDir+'*OneOverPt*Feb*tfrecord.gzip')

dataset = io.get_Classic_NN_dataset(batchSize, nEpochs, trainFileNames, isTrain=True)

print("dataset", dataset)

Reading data from files:
dataset <ParallelMapDataset element_spec=(TensorSpec(shape=(4096, 37), dtype=tf.float16, name=None), (TensorSpec(shape=(4096,), dtype=tf.float16, name=None),), TensorSpec(shape=(4096,), dtype=tf.float16, name=None))>


### Model definition

In [9]:
import model_functions as models
importlib.reload(models)

import io_functions as io
importlib.reload(io)

networkInputSize = 2 * np.sum(io.getFeaturesMask()) + 1
loss_fn = 'mae'

model = models.get_Classic_NN(networkInputSize=networkInputSize, loss_fn=loss_fn)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pt_layer_1 (Dense)          (None, 128)               4864      
                                                                 
 pt_layer_2 (Dense)          (None, 64)                8256      
                                                                 
 pt_layer_3 (Dense)          (None, 48)                3120      
                                                                 
 dense_1 (Dense)             (None, 1)                 49        
                                                                 
Total params: 16,289
Trainable params: 16,289
Non-trainable params: 0
_________________________________________________________________


### The training loop

In [11]:
from keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [12]:
%%time
current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training start. Current Time =", current_time)

nEpochs = 50

log_dir = "logs/fit/" + current_time + dir_postfix
job_dir = "training/" + current_time + dir_postfix

checkpoint_path = job_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq = 5085)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=(10, 20))
lr_reduce_callback = tf.keras.callbacks.ReduceLROnPlateau(factor=0.67, patience=3, verbose=1, min_lr=1E-5),
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, verbose=1)
callbacks =  [tensorboard_callback, cp_callback, lr_reduce_callback, early_stop_callback]

model.save_weights(checkpoint_path.format(epoch=0))
   
model.fit(dataset, epochs=nEpochs, shuffle=True,
            callbacks=callbacks
            )
model.save(job_dir, save_format='tf')

current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training end. Current Time =", current_time)

Training start. Current Time = 2023_Apr_06_22_06_22
Epoch 1/50


2023-04-06 22:06:22.075079: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-06 22:06:22.075274: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-06 22:06:22.075607: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.